In [11]:
import os
import json 

In [12]:
manifesttrain = '/home/aslin/CODE/PythonProjects/vodopetova/train/manifest.jsonl'
manifesttest = '/home/aslin/CODE/PythonProjects/vodopetova/test/crowd/manifest.jsonl'

In [13]:
def writefiles(manifest, root, number, cat):
    data = []
    with open(manifest) as f:
        for line in f:
            data.append(json.loads(line))
    if cat is None:
        cat = root
    with open(os.path.join(cat, 'audio_paths'), 'w') as paths, open(os.path.join(cat, 'text'), 'w') as texts:
        if number is not None:
            data = data[:number]
        for elem in data:
            if elem['text'].isspace():
                continue
            print(f"{elem['id']} {os.path.join(root, elem['audio_filepath'])}", file=paths)
            print(f"{elem['id']} {elem['text']}", file=texts)

In [4]:
writefiles(manifesttest, '/home/aslin/CODE/PythonProjects/vodopetova/test/crowd', 200, 'test')

In [13]:
writefiles(manifesttrain, '/home/aslin/CODE/PythonProjects/vodopetova/train', 800, 'train')

In [14]:
writefiles('/home/aslin/CODE/PythonProjects/vodopetova/validnew.jsonl', '/home/aslin/CODE/PythonProjects/vodopetova/valid_dialect', None, None)

In [23]:
writefiles('/home/aslin/CODE/PythonProjects/vodopetova/finetune.jsonl', '/home/aslin/CODE/PythonProjects/vodopetova/finetune', None, None)

In [10]:
with open('finetune_com.jsonl') as inp, open('finetune.jsonl', 'w') as out:
    for line in inp:
        l = line.split('#')[0].strip()
        if not l.isspace() and l:
            out.write(l + '\n')

In [10]:
import os

files = os.listdir('test_final')
for file in files:
    if file.endswith('channel_1.wav'):
        os.rename('test_final/' + file, 'test_final/' + file[:-14])
    # else:
        # os.remove('/home/aslin/CODE/PythonProjects/vodopetova/test_final/' + file)

In [16]:
# combine
import os, shutil 

def combine(manifest_golos, manifest_dialect, numberdial, cat):
    data_golos = []
    with open(manifest_golos) as f:
        for line in f:
            data_golos.append(json.loads(line))
    data_dialect = []
    with open(manifest_dialect) as f:
        for line in f:
            data_dialect.append(json.loads(line))
    if not os.path.exists(cat):
        os.mkdir(cat)
    with open(os.path.join(cat, 'audio_paths'), 'w') as paths, open(os.path.join(cat, 'text'), 'w') as texts:
        data = data_golos[:800 - numberdial] + data_dialect[:numberdial]
        for elem in data:
            if elem['text'].isspace():
                continue
            if elem['audio_filepath'].startswith('crowd'):
                elem['audio_filepath'] = '/home/aslin/CODE/PythonProjects/vodopetova/train/' + elem['audio_filepath']
            else:
                elem['audio_filepath'] = os.path.join('/home/aslin/CODE/PythonProjects/vodopetova/finetune', elem['audio_filepath'])
            path = os.path.join(cat, elem['audio_filepath'].split('/')[-1])
            shutil.copy(elem['audio_filepath'], path)
            print(f"{elem['id']} {os.path.join('/home/aslin/CODE/PythonProjects/vodopetova', path)}", file=paths)
            print(f"{elem['id']} {elem['text']}", file=texts)

In [5]:
os.path.join('/home/aslin/CODE/PythonProjects/vodopetova', 'finetune600x200', "1.wav")

'/home/aslin/CODE/PythonProjects/vodopetova/finetune600x200/1.wav'

In [21]:
combine(manifesttrain, '/home/aslin/CODE/PythonProjects/vodopetova/finetune.jsonl', 600, 'finetune600x200')

In [22]:
combine(manifesttrain, '/home/aslin/CODE/PythonProjects/vodopetova/finetune.jsonl', 200, 'finetune200x600')